In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# 데이터 준비

In [2]:
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 데이터 탐색

In [3]:
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [4]:
# 결측치 개수 확인
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# 결측치가 많은 deck열은 삭제
# embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(["deck", "embark_town"], axis = 1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [6]:
# age 열에 결측치가 있는 모든 행을 삭제(177개 행)
rdf = rdf.dropna(subset = ["age"], how = "any", axis = 0)

In [7]:
len(rdf)

714

In [8]:
# embarked 열의 NaN 값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf["embarked"].value_counts().idxmax()
most_freq

'S'

In [9]:
# 최빈값을 확인하는 다른 방법
rdf.describe(include = "all")

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
rdf["embarked"].fillna(most_freq, inplace = True)

# 분석에 사용할 속성을 선택

In [11]:
ndf = rdf[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [12]:
# 원핫인코딩 - 범주형데이터를 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf["sex"])

In [13]:
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [14]:
ndf = pd.concat([ndf, onehot_sex], axis = 1)

In [15]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [18]:
ndf["embarked"].unique()

array(['S', 'C', 'Q'], dtype=object)

In [16]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town")

In [17]:
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [19]:
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

In [20]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [21]:
ndf.drop(["sex", "embarked"], axis = 1, inplace = True)

In [22]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


# 데이터셋 분할 - 훈련/테스트

In [23]:
# 변수 선택
X = ndf.drop("survived", axis = 1) # 독립변수 X
Y = ndf["survived"] # 종속 변수 Y

In [24]:
# train_data와 test_data 구분(7:3 비율)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,
                                                    random_state = 4)

In [25]:
len(x_train), len(x_test)

(499, 215)

# KNN 분류

In [26]:
# KNN 분류 모형 객체 생성 (k = 5로 설정)
knn = KNeighborsClassifier(n_neighbors = 5)

In [27]:
# train_data로 모형 학습
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [28]:
# test_data로 예측
y_pred = knn.predict(x_test)

In [29]:
pd.DataFrame({"ans" : y_test, "pred" : y_pred})

,ans,pred
480,0,0
325,1,1
141,1,0
541,0,0
242,0,0
...,...,...
579,1,1
402,0,0
362,0,0
30,0,0


In [30]:
# 모형 성능 평가 - Confusion Matrix 계산
knn_matrix = metrics.confusion_matrix(y_test, y_pred)

In [31]:
# 0인 레이블을 0이라 한 경우, 0인 레이블을 1이라 한 경우
# 1인 레이블을 0이라 한 경우, 1인 레이블을 1이라 한 경우
print(knn_matrix)

[[109  15]
 [ 38  53]]


In [35]:
y_test.value_counts()

0    124
1     91
Name: survived, dtype: int64

In [36]:
# 모형 성능 평가 - 평가 지표 계산
knn_report = metrics.classification_report(y_test, y_pred)

In [37]:
print(knn_report)

              precision    recall  f1-score   support

           0       0.74      0.88      0.80       124
           1       0.78      0.58      0.67        91

    accuracy                           0.75       215
   macro avg       0.76      0.73      0.74       215
weighted avg       0.76      0.75      0.75       215



- precision(정밀도)
    - 양성으로 예측한 데이터 중 실제로 양성인 데이터의 비율
    
- recall(재현율)
    - 실제 양성인 데이터 중 양성으로 예측한 비율
    
- 코로나 검사 키트의 경우
    - 민감도 90% 이상
        - 실제 양성인 사람이 코로나 검사를 하면 양성으로 예측될 확률 90% 이상
    - 특이도 99% 이상
        - 실제 음성인 사람이 코로나 검사를 하면 음성으로 예측될 확률 99% 이상

# 모델 고도화

In [38]:
ss = preprocessing.StandardScaler()

In [39]:
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [41]:
scaled_test

array([[ 0.90434532, -1.41237825,  4.55136791, ..., -0.46272229,
        -0.21475938,  0.53176719],
       [-1.45831601,  0.43352665, -0.55919147, ...,  2.16112345,
        -0.21475938, -1.88052217],
       [ 0.90434532, -0.52360922, -0.55919147, ..., -0.46272229,
        -0.21475938,  0.53176719],
       ...,
       [ 0.90434532,  1.04882829, -0.55919147, ...,  2.16112345,
        -0.21475938, -1.88052217],
       [-1.45831601,  0.70699405, -0.55919147, ...,  2.16112345,
        -0.21475938, -1.88052217],
       [ 0.90434532, -0.93381031, -0.55919147, ..., -0.46272229,
        -0.21475938,  0.53176719]])

In [42]:
knn = KNeighborsClassifier(n_neighbors = 5)

In [43]:
knn.fit(scaled_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [44]:
y_pred = knn.predict(scaled_test)

In [45]:
pd.DataFrame({"ans" : y_test, "pred" : y_pred})

,ans,pred
480,0,0
325,1,1
141,1,0
541,0,0
242,0,0
...,...,...
579,1,1
402,0,0
362,0,1
30,0,1


In [46]:
knn_matrix = metrics.confusion_matrix(y_test, y_pred)
print(knn_matrix)

[[109  15]
 [ 26  65]]


In [47]:
knn_report = metrics.classification_report(y_test, y_pred)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.88      0.84       124
           1       0.81      0.71      0.76        91

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215

